## Pedigree Package 


### Install PedModule

Do this only once

In [1]:
#Pkg.clone("https://github.com/reworkhow/PedModule.jl.git")

INFO: Cloning PedModule from https://github.com/reworkhow/PedModule.jl.git
INFO: Computing changes...


In [2]:
using PedModule

### Pedigree file

In [3]:
;cat ex1.ped

AA93 A1 AA92
A1   C2 223G
FFG1 C2 C99
AA92 A1 FFG1

### Assign sequential IDs and compute inbreeding

In [4]:
ped = PedModule.mkPed("ex1.ped")
ped.idMap

Dict{Any,Any} with 7 entries:
  "FFG1" => PedNode(3,"C2","C99",0.0)
  "C2"   => PedNode(1,"0","0",0.0)
  "AA92" => PedNode(6,"A1","FFG1",0.125)
  "C99"  => PedNode(2,"0","0",0.0)
  "AA93" => PedNode(7,"A1","AA92",0.3125)
  "A1"   => PedNode(5,"C2","223G",0.0)
  "223G" => PedNode(4,"0","0",0.0)

### Get inverse of additive relationship matrix

In [34]:
Ai = PedModule.AInverse(ped)

7x7 sparse matrix with 29 Float64 entries:
	[1, 1]  =  2.0
	[2, 1]  =  0.5
	[3, 1]  =  -1.0
	[4, 1]  =  0.5
	[5, 1]  =  -1.0
	[1, 2]  =  0.5
	[2, 2]  =  1.5
	[3, 2]  =  -1.0
	[1, 3]  =  -1.0
	[2, 3]  =  -1.0
	⋮
	[4, 5]  =  -1.0
	[5, 5]  =  3.03333
	[6, 5]  =  -0.466667
	[7, 5]  =  -1.06667
	[3, 6]  =  -1.0
	[5, 6]  =  -0.466667
	[6, 6]  =  2.53333
	[7, 6]  =  -1.06667
	[5, 7]  =  -1.06667
	[6, 7]  =  -1.06667
	[7, 7]  =  2.13333

In [35]:
full(Ai)

7x7 Array{Float64,2}:
  2.0   0.5  -1.0   0.5  -1.0        0.0        0.0    
  0.5   1.5  -1.0   0.0   0.0        0.0        0.0    
 -1.0  -1.0   2.5   0.0   0.5       -1.0        0.0    
  0.5   0.0   0.0   1.5  -1.0        0.0        0.0    
 -1.0   0.0   0.5  -1.0   3.03333   -0.466667  -1.06667
  0.0   0.0  -1.0   0.0  -0.466667   2.53333   -1.06667
  0.0   0.0   0.0   0.0  -1.06667   -1.06667    2.13333

### Additive relationship matrix

In [36]:
A = round(inv(full(Ai)),3)

7x7 Array{Float64,2}:
  1.0  -0.0    0.5    0.0    0.5    0.5    0.5  
 -0.0   1.0    0.5    0.0    0.0    0.25   0.125
  0.5   0.5    1.0    0.0    0.25   0.625  0.438
  0.0  -0.0    0.0    1.0    0.5    0.25   0.375
  0.5   0.0    0.25   0.5    1.0    0.625  0.813
  0.5   0.25   0.625  0.25   0.625  1.125  0.875
  0.5   0.125  0.438  0.375  0.813  0.875  1.313

In [22]:
n = length(ped.idMap)
a = Array(Int64,n,3);
f = Array(Float64,n);

In [24]:
for i in keys(ped.idMap)
    indSeqID  = ped.idMap[i].seqID
    sire      = ped.idMap[i].sire
    if   sire!= "0"
        sireSeqID = ped.idMap[sire].seqID
    else
        sireSeqID = 0
    end
    dam       = ped.idMap[i].dam
    if dam   != "0"
        damSeqID = ped.idMap[dam].seqID
    else
        damSeqID = 0
    end
    a[indSeqID,1] = indSeqID
    a[indSeqID,2] = sireSeqID
    a[indSeqID,3] = damSeqID
    f[indSeqID]   = ped.idMap[i].f
end

In [31]:
for i=1:7
    @printf("%3d %3d %3d %6.3f \n", a[i,1], a[i,2], a[i,3], f[i])
end

  1   0   0  0.000 
  2   0   0  0.000 
  3   1   2  0.000 
  4   0   0  0.000 
  5   1   4  0.000 
  6   5   3  0.125 
  7   5   6  0.313 


In [37]:
keys(ped.idMap)

KeyIterator for a Dict{Any,Any} with 7 entries. Keys:
  "FFG1"
  "C2"
  "AA92"
  "C99"
  "AA93"
  "A1"
  "223G"